In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import numpy as np
from datetime import date

In [2]:
today = date.today()
date = today.strftime("%d/%m/%Y")

In [3]:
date

'10/06/2022'

In [4]:
# quantidade de páginas do site
n_pages = 130

# data para etiqueta do csv
info = time.localtime(time.time())
# etiqueta do csv com data de execução do código
label = f'\dados_{date}.csv'

# lista que vai armazenar o conteúdo de cada página
pages = []
for i in range(1,n_pages + 1):
    r = requests.get(f'https://www.coingecko.com/pt?page={i}', auth=('user', 'pass'))

    bs = BeautifulSoup(r.content,'html')
    table = bs.find_all('table')[0]
    pages.append(pd.read_html(str(table))[0])

In [5]:
# junta as páginas em um dataframe
df = pd.concat(pages)

In [6]:
#Limpeza de dados
df = df.drop('Unnamed: 0',1)
df = df.drop('Unnamed: 3', 1)
df = df.drop('Últimos 7 dias',1)
df.dropna(axis=0,subset=['Preço', '1 h', '24 h', '7 d'], inplace=True)

df['#'] = range(1,len(df)+1)
df = df[df['1 h'] != '?']
df = df[df['24 h'] != '?']
df = df[df['7 d'] != '?']
df = df[df['Capitalização de Mercado'] != '?']

df['#'] = range(1,len(df)+1)
df['Preço'] = df['Preço'].str.replace('$','')
df['Capitalização de Mercado'] = df['Capitalização de Mercado'].str.replace('$','')
df['Volume em 24 h'] = df['Volume em 24 h'].str.replace('$','')
lista = ['1 h', '24 h', '7 d']
for i in lista: df[f'{i}'] = df[f'{i}'].str.replace('%','')
s = df.columns[2:]
for i in s: df[f'{i}'] = df[f'{i}'].str.replace('.','')
for i in s: df[f'{i}'] = df[f'{i}'].str.replace(',','.')
for i in s: df[f'{i}'] = df[f'{i}'].astype(float)
df['7 d'] = df['7 d']/100
df['1 h'] = df['1 h']/100
df['24 h'] = df['24 h']/100

df = df.assign(Data = date)

df_completo = pd.read_csv('final.csv', sep = ' ')
df = pd.concat([df_completo, df])
df.to_csv('final.csv', sep = ' ', index=False)


<ipython-input-6-0ac5610e7123>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Preço'] = df['Preço'].str.replace('$','')
<ipython-input-6-0ac5610e7123>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Capitalização de Mercado'] = df['Capitalização de Mercado'].str.replace('$','')
<ipython-input-6-0ac5610e7123>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Volume em 24 h'] = df['Volume em 24 h'].str.replace('$','')
<ipython-input-6-0ac5610e7123>:20: FutureWarning: The default value of regex will change fro

In [7]:
df

,#,Moeda,Preço,1 h,24 h,7 d,Volume em 24 h,Capitalização de Mercado,Data
0,1,Bitcoin BTC BTC,2.837640e+04,-0.04,-0.40,-0.66,3.112345e+10,5.396522e+11,27/05/2022
1,2,Ethereum ETH ETH,1.727410e+03,-0.08,-0.74,-1.46,2.397160e+10,2.094191e+11,27/05/2022
2,3,Tether USDT USDT,1.000000e+00,0.02,0.01,0.00,5.923718e+10,7.276477e+10,27/05/2022
3,4,USD Coin USDC USDC,1.000000e+00,0.02,-0.00,-0.01,6.175088e+09,5.354745e+10,27/05/2022
4,5,BNB BNB BNB,2.984500e+02,-0.04,-0.46,-0.30,2.868084e+09,5.028798e+10,27/05/2022
...,...,...,...,...,...,...,...,...,...
70,3229,Coin Fast Alert [OLD] CFA CFA,1.800000e-09,0.58,0.58,2.85,1.550000e+00,1.550966e-02,10/06/2022
73,3230,FrogeX FROGEX FROGEX,9.795940e-07,0.05,-3.63,-4.78,6.861310e+03,4.015775e-04,10/06/2022
74,3231,Fesschain FESS FESS,1.000000e-12,0.00,0.00,0.00,0.000000e+00,3.121681e-04,10/06/2022
77,3232,VersoView VVT VVT,0.000000e+00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,10/06/2022
